In [1]:
from pathlib import Path
from ast import literal_eval
from collections import  namedtuple
from itertools import product, takewhile
from tqdm import tqdm
import re

In [2]:
# Token types
Choice = namedtuple("Choice", "s")
Sequence = namedtuple("Sequence", "s")
Constant = namedtuple("Constant", "s")
SubRule = namedtuple("SubRule", "s")

def read_rules(text_rules, problem=1):
    
    patterns = {}
    unsolved = {}
    PROBLEM2_RULEIDS = (8,11)
    
    def make_sequence(chunk):
        "A sequence of one item is collapsed to a sub-rule"
        words = chunk.split()
        if len(words) == 1:
            return SubRule(int(words[0]))
        else:
            return Sequence([SubRule(int(w)) for w in words])
    
    for line in text_rules.splitlines():
        ruleid, ruledata = line.split(":")
        ruleid = int(ruleid)
        
        if (problem == 2) and (ruleid in PROBLEM2_RULEIDS):
            continue
        
        if '"' in ruledata:
            patterns[ruleid] = Constant(literal_eval(ruledata.strip()))
            
        elif "|" in ruledata:
            chunks = ruledata.split("|")
            subrules = [ make_sequence(chunk) for chunk in chunks ] 
            unsolved[ruleid] = Choice(subrules)
            
        else: 
            unsolved[ruleid] = make_sequence(ruledata)
            
    if problem == 2:
        # stash the simplified (exactly one repeat) versions of the rules for now
        unsolved[8] = SubRule(42)
        unsolved[11] = Sequence([SubRule(42), SubRule(31)])
            
    def resolver(item):
        if type(item) is Constant:
            return item
        elif type(item) is Sequence:
            assert len(item.s) > 1
            return Sequence([resolver(ii) for ii in item.s])
        elif type(item) is Choice:
            assert len(item.s) == 2
            return Choice([resolver(ii) for ii in item.s])
        elif type(item) is SubRule:
            return patterns[item.s]
        else:
            raise TypeError(f"What the heck??? {item} {type(item)}")
    
    while len(unsolved):
        for key, value in unsolved.copy().items():
            try:
                if type(value) is SubRule:
                    patterns[key] = resolver(patterns[value.s])
                else:
                    patterns[key] = resolver(value)
                unsolved.pop(key)
            except KeyError:
                    pass
                
    return patterns

def yield_pattern(rule):
    if type(rule) is Constant:
        yield rule.s
        
    elif type(rule) is Choice:
        yield from yield_pattern(rule.s[0])
        yield from yield_pattern(rule.s[1])
        
    elif type(rule) is Sequence:
        iterators = (yield_pattern(item) for item in rule.s)
        for parts in product(*iterators):
            yield "".join(parts)
        
    else:
        raise ValueError("Huh, interesting")

# Problem 1

In [3]:
data = Path("rules.txt").read_text()
ruledata, imagedata = data.split("\n\n")
my_rules = read_rules(ruledata)
valid_patterns = set(yield_pattern(my_rules[0]))

match = sum([line in valid_patterns for line in imagedata.splitlines() ])
match

222

# Problem 2

Completely replace rules 8: 42 and 11: 42 31 with the following:

```
8: 42 | 42 8
11: 42 31 | 42 11 31
```

This small change has a big impact: now, the rules do contain loops, and the list of messages they could hypothetically match is infinite. 
Fortunately, many of the rules are unaffected by this change; it might help to start by looking at which rules always match the same set of values and how those rules (especially rules 42 and 31) are used by the new versions of rules 8 and 11.

(Remember, you only need to handle the rules you have; building a solution that could handle any hypothetical combination of rules would be significantly more difficult.)

## My approach

So,  both rule 42 and rule 31 are going to match 8 character sequences, and the sequences in each rule 42 and 31 do not overlap.

The matching sequences should be a sequence of 8 character chunks in set 42, followed by a sequence of 8 character chunks 
in set 31. There need to be more set 42 entries than set 31 entries. (For rule 11, the number of 42's must equal the number of 31's; for
rule 8, there are 1 or more 42's.)


In [4]:
my_rules = read_rules(ruledata, problem=2)

set_42 = set(yield_pattern(my_rules[42]))
set_31 = set(yield_pattern(my_rules[31]))

print( set([len(x) for x in set_42]), set([len(x) for x in set_31]))
print( set_42.intersection(set_31))

{8} {8}
set()


In [5]:
match = 0
chunksize = 8

for line in imagedata.splitlines():
    if len(line) % chunksize != 0:
        continue
    
    # state machine states named after the set we're searching for,
    # or -1 for "never gonna match now"
    state = 42
    count_42 = 0
    count_31 = 0
    
    chunks = [line[i:i+chunksize] for i in range(0,len(line),chunksize)]
    for chunk in chunks:
        if state == 42:
            if chunk in set_42:
                count_42 += 1
            elif chunk in set_31:
                state = 31
                count_31 += 1
            else:
                state = -1 # nomatch flag
        elif state == 31:
            if chunk in set_31:
                count_31 += 1
            else:
                state = -1
    if (state == 31) and (count_42 > count_31):
        match += 1
        
    
match

339

In [6]:
# Test for part 1

example_rules = '''0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"'''
example_data = '''ababbb
bababa
abbbab
aaabbb
aaaabbb'''

example_rules = read_rules(example_rules)
matching_patterns = list(yield_pattern(example_rules[0]))
print(matching_patterns) # should be 8 in the toy example

test = 0
for line in example_data.splitlines():
    for pat in matching_patterns:
        if line == pat:
            test += 1
test

['aaaabb', 'aaabab', 'abbabb', 'abbbab', 'aabaab', 'aabbbb', 'abaaab', 'ababbb']


2